# Simple ant colony optimization (S-ACO)
- We have the ability to construct a path with a single ant 
- Lets make a number of ants work together to find the shortest path
- Now we introduce alpha to our probability to control the pheromone influence
- Pheromone evaporation is also introduced in the update loop 
- The Algorithm is reconstructed with the modifications to get a simple ant colony algorithm

# 1.1 Define Choice Mode
- how do i make my next move
- after making all moves i have to remove loops using a scanning strategy

In [1]:
import random

def NextNode(Connections, RouletteProb):
    
    """ takes a list of connections and corresponding Roulette probabilities
    and returns an one connetion from the list of connetions 
    
    connections: List of nodes
    RouletteProb: List corresponding node roulette probability
    returns: int Node choosen """
    
    r = random.random()
    for i in range(len(Connections)):
        if i == 0 and r <= RouletteProb[i]:
            return Connections[i]
        elif r > RouletteProb[i] and r <= RouletteProb[i+1]:
            return Connections[i+1]
        
# Eliminate Loops 
def Deloop(route):
    l = len(route)
    for i in range(l):
        for j in range(i+1,l):
            j = l+i-j
            if j < len(route) and route[i] == route[j]:          # list changes as loop is removed 
                # print("Delooping:  ", x)
                [route.remove(k) for k in route[i+1:j+1]]
                # print("Delooped to: ", x)
    return route

# 1.2 Introduce alpha
- control influence of pheromone on probability 

In [2]:
def Travel(costGraph, pheromoneGraph, source, destination, alpha = 1):
    """ takes a 
    - connection graph (costGraph) a square matrix, 
    - pheromone graph (pheromoneGraph) a square matrix same size as costGraph, 
    - takeoff point (source) and destination are the int Node number
    - alpha balances the pheromone influence 
    Returns a complete tour as a List
    """
    
    route = [source]
    while source != destination:  
        Connections = [i for i,j in enumerate(costGraph[source]) if j>0]    
        pheromoneIJS = [ pheromoneGraph[source][i]**alpha for i in Connections]
        ProbabilityIJS = [ i/sum(pheromoneIJS) for i in pheromoneIJS]
        RouletteProb = [ sum(ProbabilityIJS[:i+1]) for i in range(len(ProbabilityIJS)) ]
        source = NextNode(Connections, RouletteProb)
        route.append(source)
    
    return Deloop(route)

# 1.3 Introduce evoparation of pheromone rho

- removes a proportion of pheromone from paths after every iteration 

In [3]:
def Updatepheromone(route, pheromoneGraph, rho = 0.5):
    """ takes a route and use it to add pheromone to arcs in the 
    route
    
    route : route taken
    pheromoneGraph: pheromone matrix
    rho : evaporation rate 
    returns pheromone graph """
    
    tau = 1/len(route)
    # tau = 1
    for i in range(len(route)-1):
        pheromoneGraph[route[i]][route[i+1]] = (1-rho)*(pheromoneGraph[route[i]][route[i+1]]) + tau
        pheromoneGraph[route[i+1]][route[i]] = (1-rho)*(pheromoneGraph[route[i+1]][route[i]]) + tau

    return pheromoneGraph

# 1.4 Finally Define the Algorithm 
- Use to find the shortest path 

In [4]:
def SACO(costGraph, takeoff, destination, Population = 8, alpha = 2, rho = 0, iterations = 100):
    
    """ takes a costgraph, takeoff point, destination point and number of times to travel (iterations)
    and return a tour """
    length = len(costGraph)
    PRM = [[1/length for i in range(length)] for j in range(length)]
    Ants = {"Ant" + str(i+1): None for i in range(Population)}
    
    while iterations > 0:
        #--------------TRAVEL-----------------------------------------
        for ant in Ants:
            Ants[ant] = Travel(costGraph, PRM, takeoff, destination, alpha)
        #-------------UPDATE PHEROMONE--------------------------------
        for ant in Ants:
            PRM = Updatepheromone(Ants[ant], PRM, rho)
        #-------------------------------------------------------------
        iterations -= 1
        #print(Route)
    return {"Ants": Ants, "PRM":PRM}

# 1.5 Solve the Double bridge Path problem

In [5]:
EDBM = [[0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
        [1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0],
        [1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0],
        [0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0],
        [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1],
        [0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0]]

iterations = 11
takeoff = 0
destination = 8
population = 8
alpha = 2
rho = 0

Tour = SACO(EDBM, takeoff, destination, population, alpha, rho, iterations)
Tour["Ants"]

{'Ant1': [0, 9, 14, 17, 13, 8],
 'Ant2': [0, 9, 14, 17, 13, 8],
 'Ant3': [0, 9, 14, 17, 13, 8],
 'Ant4': [0, 9, 14, 17, 13, 8],
 'Ant5': [0, 9, 14, 17, 13, 8],
 'Ant6': [0, 9, 14, 17, 13, 8],
 'Ant7': [0, 9, 14, 17, 13, 8],
 'Ant8': [0, 9, 14, 17, 13, 8]}

# 1.6 Visualize Pheromone Intensity on Paths 

In [6]:
import numpy as np
import matplotlib.pyplot as plt

data = np.array(Tour["PRM"])
hm = plt.imshow(data, 'gray', origin='lower')

plt.colorbar(hm)

# 1.7 TODO

- use the matrix provided to solve the shortest path problem 
- display the pheromone intensity map 
<img src="images/shortbridge.png" width="400"/>

- The matrix is given below

In [7]:
# Shortest path is [ 0,  3,  4]

CIN = [[0,0,0,1,0,1,0],
       [0,0,1,1,0,0,0],
       [0,1,0,1,1,0,0],
       [1,1,1,0,1,1,0],
       [0,0,1,1,0,0,1],
       [1,0,0,1,0,0,1],
       [0,0,0,0,1,1,0]]